<b> First, check to make sure MegaDetector is available.
<br> CameraTraps, ai4eutils, and yolo5 must be included in the PYTHONPATH

In [1]:
import pandas as pd
import sys
sys.path

['C:\\Users\\tswanson\\animl-py\\src\\animl',
 'C:\\Users\\tswanson\\animl-py\\src\\animl\\%PYTHONPATH%',
 'C:\\Users\\tswanson\\CameraTraps',
 'C:\\Users\\tswanson\\ai4eutils',
 'C:\\Users\\tswansont\\yolov5',
 'C:\\Users\\tswanson\\yolov5',
 'C:\\Users\\tswanson\\anaconda3\\python39.zip',
 'C:\\Users\\tswanson\\anaconda3\\DLLs',
 'C:\\Users\\tswanson\\anaconda3\\lib',
 'C:\\Users\\tswanson\\anaconda3',
 '',
 'C:\\Users\\tswanson\\anaconda3\\lib\\site-packages',
 'C:\\Users\\tswanson\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg',
 'C:\\Users\\tswanson\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\tswanson\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\tswanson\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\tswanson\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\tswanson\\.ipython']

<b> Find file paths for the image directory of interest, both model files, and the classifier class list.

In [2]:
imagedir = "/home/kyra/animl-py/examples/Southwest"
model_file = "/mnt/machinelearning/megaDetector/md_v5b.0.0.pt"
class_model = "/mnt/machinelearning/Models/Southwest/2022/EfficientNetB5_456_Unfrozen_05_0.26_0.92.h5"
classes = "/mnt/machinelearning/Models/Southwest/2022/classes.txt"

In [3]:
imagedir = "C:\\Users\\tswanson\\animl-py\\examples\\Southwest"
model_file = "C:\\Users\\tswanson\models\\\md_v5b.0.0.pt"
class_model = "C:\\Users\\tswanson\\models\\2022\\Southwest2.0.h5"
classes = "C:\\Users\\tswanson\\models\\2022\\classes.txt"

<b> Create a working directory and build the file manifest from the image directory.

In [4]:
import fileManagement
workingdir = fileManagement.WorkingDirectory(imagedir)

In [5]:
files = fileManagement.build_file_manifest(imagedir, out_file = workingdir.filemanifest)

<b> If there are videos, save out individual frames to pass through models. 

In [6]:
import videoProcessing
allframes = videoProcessing.images_from_videos(files, out_dir=workingdir.vidfdir, 
                                               out_file=workingdir.imageframes, parallel = True, frames=2)

<b> Run all images and video frames through MegaDetector

In [7]:
import detectMD
mdresults = detectMD.detect_MD_batch(model_file, allframes["Frame"], checkpoint_path=None,
                                checkpoint_frequency=-1, results=None, n_cores=1, quiet=True)

PyTorch reports 0 available CUDA devices
GPU available: False
Using PyTorch version 1.10.1+cpu


Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients, 207.9 GFLOPs


Loaded model in 4.45 seconds
Loaded model in 4.46 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:56<00:00,  4.38s/it]


<b> Convert MD JSON to pandas dataframe, merge with manifest.

In [8]:
import parseResults
mdres = parseResults.parseMD(mdresults, manifest=allframes, out_file = workingdir.mdresults)

<b> Extract animal detections from the rest.

In [9]:
import splitData
animals = splitData.getAnimals(mdres)
empty = splitData.getEmpty(mdres)

<b> Use the classifier model to predict the species of animal detections.

In [10]:
import predictSpecies
predresults = predictSpecies.predictSpecies(animals, class_model, batch = 4)

6
6/6 [==============================] - 11s 1s/step


In [11]:
import parseResults
animals = parseResults.applyPredictions(animals,predresults,classes, out_file = workingdir.predictions)

In [14]:
manifest = pd.concat([animals,empty])

In [29]:
manifest

,FilePath,FileName,FileModifyDate,Frame,file,...,bbox2,bbox3,bbox4,prediction,confidence
0,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-05-28_SYFR0041.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.43780,0.34300,0.51880,deer,0.940423
1,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-06-30_SYFR0218.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.37130,0.24050,0.25360,cougar,0.967000
2,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-07-24_SYFR0655.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.45070,0.23480,0.21230,coyote,0.971547
3,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-08-08_RCNX0063.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.57860,0.18750,0.12800,fox,0.968689
4,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-11-11_11110007.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.02000,0.89770,0.94560,bird,0.010882
5,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-11-11_11110007.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.02000,0.90140,0.40100,bird,0.881811
6,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-11-14_11150025.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.54190,0.41630,0.40660,bobcat,0.974207
7,C:\Users\tswanson\animl-py\examples\Southwest\...,2021-12-07_12080069.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.48560,0.05416,0.07638,rabbit,0.932372
8,C:\Users\tswanson\animl-py\examples\Southwest\...,IMAG0004.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.38330,0.07773,0.05468,deer,0.005718
9,C:\Users\tswanson\animl-py\examples\Southwest\...,IMAG0004.JPG,2023-07-07 13:18:05,C:\Users\tswanson\animl-py\examples\Southwest\...,C:\Users\tswanson\animl-py\examples\Southwest\...,...,0.40310,0.24720,0.11140,cougar,0.941000


In [32]:
"""
Module: animl.plot_boxes
Functionality to draw bounding boxes and labels provided image DataFrame.
"""

import cv2


def demo_boxes(manifest, min_conf = 0.9, prediction = True):
    # pylint: disable=R0914
    """
    Draws bounding boxes and labels on image DataFrame.
    Args:
        image : DataFrame containing image data - coordinates and predictions.
            The DataFrame should have the following columns:
            - 'Frame': Filename or path to the image file.
            - 'bbox1': Normalized x-coordinate of the top-left corner.
            - 'bbox2': Normalized y-coordinate of the top-left corner.
            - 'bbox3': Normalized width of the bounding box (range: 0-1).
            - 'bbox4': Normalized height of the bounding box (range: 0-1).
            - 'prediction': Object prediction label for the bounding box.
        n (int): Number used for generating the output image filename.
        imageOutputPath (str): Output directory to saved images.
    Returns:
        None
    """
    images = manifest["FilePath"].unique()
    
    for image_path in images:
        #display the image, wait for key
        img = cv2.imread(image_path)  # Use row directly without indexing
        cv2.namedWindow("Display", cv2.WINDOW_NORMAL)
        cv2.imshow('Display', img)
        cv2.waitKey(0)
        
        boxes = manifest[manifest["FilePath"] == image_path]
        print(boxes)
        for _,row in boxes.iterrows():
            confidence = row["confidence"]
            if confidence >= min_conf:
                height, width, _ = img.shape
                left = int(row['bbox1'] * width)
                top = int(row['bbox2'] * height)
                right = int((row['bbox1'] + row['bbox3']) * width)
                bottom = int((row['bbox2'] + row['bbox4']) * height)
                label = row['prediction']
                text_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_DUPLEX, 1, 1)
                text_size_width, text_size_height = text_size
                thick = int((height + width) // 900)
                box_right = (right if (right - left) < (text_size_width * 3)
                            else left + (text_size_width * 3))

                cv2.rectangle(img, (left, top), (right, bottom), (90, 255, 0), thick)

                if prediction:

                    cv2.rectangle(img, (left, top),
                              (box_right, top - (text_size_height * 3)),
                              (90, 255, 0), -1)
                    cv2.putText(img, label, (left, top - 12), 0, 1e-3 * height,
                                (0, 0, 0), thick // 3)
                cv2.imshow('Display', img)
                cv2.waitKey(0)
            
            else:
                continue

    cv2.destroyAllWindows()

In [33]:
demo_boxes(manifest)

                                            FilePath                 FileName  \
0  C:\Users\tswanson\animl-py\examples\Southwest\...  2021-05-28_SYFR0041.JPG   

        FileModifyDate                                              Frame  \
0  2023-07-07 13:18:05  C:\Users\tswanson\animl-py\examples\Southwest\...   

                                                file  ...   bbox2  bbox3  \
0  C:\Users\tswanson\animl-py\examples\Southwest\...  ...  0.4378  0.343   

    bbox4  prediction  confidence  
0  0.5188        deer    0.940423  

[1 rows x 14 columns]
                                            FilePath                 FileName  \
1  C:\Users\tswanson\animl-py\examples\Southwest\...  2021-06-30_SYFR0218.JPG   
0  C:\Users\tswanson\animl-py\examples\Southwest\...  2021-06-30_SYFR0218.JPG   

        FileModifyDate                                              Frame  \
1  2023-07-07 13:18:05  C:\Users\tswanson\animl-py\examples\Southwest\...   
0  2023-07-07 13:18:05  C:\Users\tsw

                                             FilePath      FileName  \
16  C:\Users\tswanson\animl-py\examples\Southwest\...  IMAG1065.JPG   
17  C:\Users\tswanson\animl-py\examples\Southwest\...  IMAG1065.JPG   
18  C:\Users\tswanson\animl-py\examples\Southwest\...  IMAG1065.JPG   
19  C:\Users\tswanson\animl-py\examples\Southwest\...  IMAG1065.JPG   
20  C:\Users\tswanson\animl-py\examples\Southwest\...  IMAG1065.JPG   

         FileModifyDate                                              Frame  \
16  2023-07-07 13:18:06  C:\Users\tswanson\animl-py\examples\Southwest\...   
17  2023-07-07 13:18:06  C:\Users\tswanson\animl-py\examples\Southwest\...   
18  2023-07-07 13:18:06  C:\Users\tswanson\animl-py\examples\Southwest\...   
19  2023-07-07 13:18:06  C:\Users\tswanson\animl-py\examples\Southwest\...   
20  2023-07-07 13:18:06  C:\Users\tswanson\animl-py\examples\Southwest\...   

                                                 file  ...   bbox2    bbox3  \
16  C:\Users\tswanson\ani